# Classificador de Raças de Cachorros usando Tensorflow e Keras


Neste notebook iremos implementadar um modelo para classificação de imagens. Classificação é uma das "tarefas" em que podemos utilizar Machine Learning, nesta tarefa o ensino é **supervisionado**, em outras palavras nós vamos ensinar ao modelo através de exemplos com gabarito.

Nosso modelo deverá receber imagens de veículos e não-veículos e identificar a que **classe** (raça de cachorro) o cachorro pertence.

## Dados

Os dados são oriundos da competição [Dog Breed Indentification do Kaggle](https://www.kaggle.com/c/dog-breed-identification), na qual fornece aproximadamente 10 mil imagens de cachorros de 120 classes.


## Modelo

Iremos utilizar a [arquitetura da  InceptionV3](https://arxiv.org/abs/1512.00567), ela está implementada no [Keras](https://keras.io/applications/#inceptionv3)


## Conseguindo os dados

Para ter acesso aos dados é necessário uma conta no Kaggle, e ter que entrar na [competição](https://www.kaggle.com/c/dog-breed-identification), e ir na aba Data na competição a baixá-los

### Avisos

#### Aviso #1
Para fazer o treinamento da InceptionV3 é necessário um grande poder computacional, na qual a maioria das pessoas não possuem. Mas não será por isso que não utilizaremos a Inception, graças ao Kaggle, temos a opção de rodar Kernels (que são muito similares aos notebooks do jupyter) na infraestrutura do próprio Kaggle, para mais informações sobre o suporte a GPU's do Kaggle veja [esse notebook](https://www.kaggle.com/dansbecker/running-kaggle-kernels-with-a-gpu) do [Dan Becker](https://twitter.com/dan_s_becker)

#### Aviso #2
Esse notebook não foi executado na minha máquina, eu rodei ele nos kernels do Kaggle. Por isso não temos as saídas das células, se você quiser visualizar as saídas clique [aqui](https://www.kaggle.com/igorslima/inception)

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

np.random.seed(0)

In [ ]:
input_folder = '/kaggle/input'

In [ ]:
# lendo input
df_train = pd.read_csv(input_folder+'/labels.csv')
df_test = pd.read_csv(input_folder+'/sample_submission.csv')
df_train.breed.value_counts().plot(kind='bar', figsize=(15,15), title="Quantidade de imagens por raça no treino");

In [ ]:
df_train.head()

In [ ]:
df_test.head()

## Transormando os dados para a "notação" one-hot-encoding

Para mais informações sobre o One Hot Enconding leia este [post](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f)


In [ ]:
targets_series = pd.Series(df_train['breed'])
one_hot = pd.get_dummies(targets_series, sparse = True)
one_hot_labels = np.asarray(one_hot)

In [ ]:
im_size = 224

## Lendo as imagens

Para treinar a rede é necessário peger as imagens do disco e colocar elas em memória. Não entendeu um 'a' do que eu disse? Tudo bem, é normal. O que eu quis dizer foi que vamos ter que pegar as imagens do HD e colocar elas na memória RAM.

In [ ]:
from tqdm import tqdm # bliblioteca para colocar a porcentagem de andamento do for
import cv2 # biblioteca para visão computacional

In [ ]:
x_train = []
y_train = []
x_test = []

In [ ]:
i = 0 
for f, breed in tqdm(df_train.values):
    img = cv2.imread(input_folder+'/train/{}.jpg'.format(f))
    x_train.append(cv2.resize(img, (im_size, im_size)))
    label = one_hot_labels[i]
    y_train.append(label)
    i += 1

In [ ]:
del df_train # apagando uma variável pra diminuir consumo de memória

In [ ]:
for f in tqdm(df_test['id'].values):
    img = cv2.imread(input_folder+'/test/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (im_size, im_size)))

## Dividindo dataset

Geralmente em dividimos os dados em treino, validação e teste.
1. Treino: conjunto para treinar o modelo
2. Validação: conjunto para escolher os melhores hiperparâmetros do modelo (mais tarde falo sobre hiperparâmetros, ok?)
3. Teste: conjunto para coletar as métricas finais do modelo

In [ ]:
from sklearn.model_selection import train_test_split # biblioteca para fazer a divisão dos dados em treino e teste

In [ ]:
num_class = 120
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train, y_train, shuffle=True,  test_size=0.2, random_state=1)

## Data augmentation

Nós temos dados o suficiente para travar nossas máquinas XD, mas não o suficiente para treinar modelos bastantes robustos, temos poucas imagens por classe.

Para ameninzar esse problema iremos utilizar uma técnica chamada data augmentations, ela transforma uma imagem em diversas, como por exemplo dar um giro vertical, ou horizontal. Como nesse exemplo:

![Imgur](https://i.imgur.com/GJGMou5.png)

Links legais (em inglês, desculpem):

[Link para a documentação](https://keras.io/preprocessing/image/)

[Tutorial massa do keras](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)

[Outro tutorial massa, mas não é do Keras, esse é do Dan Becker](https://www.kaggle.com/dansbecker/data-augmentation)[](http://)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator # biblioteca para data augmetantaion

In [ ]:
datagen = ImageDataGenerator(width_shift_range=0.2, 
                            height_shift_range=0.2,
                            zoom_range=0.2,
                            rotation_range=30,
                            vertical_flip=False,
                            horizontal_flip=True) # aqui eu defino os parâmetros que irei 
                                                  # utilizar para gerar as imagens
#Aqui a ideia é rotacionar, inverter ... as fotos para que aumente o tamanho do Dataset e diminua a chance 
# de overfittar o modelo, um gato invertido continua sendo um gato

train_generator = datagen.flow(np.array(X_train), np.array(Y_train), 
                               batch_size=32) 
valid_generator = datagen.flow(np.array(X_valid), np.array(Y_valid), 
                               batch_size=32) 

## Criação da Inception

A partir de agora iremos criar a rede propriamente dita, iremos utilizar a arquitetura da rede Inception, e os pesos pré-treinada sobre os dados do ImageNet.

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, Dropout, Flatten
from keras import regularizers
from keras.models import Model

In [ ]:
base_model = InceptionV3(weights="imagenet",include_top=False, input_shape=(im_size, im_size, 3))
dropout = base_model.output
dropout = Dropout(0.5)(dropout)
model_with_dropout = Model(inputs=base_model.input, outputs=dropout)
    
x = model_with_dropout.output
x = Flatten()(x)
predictions = Dense(num_class, activation='softmax',
                    kernel_regularizer=regularizers.l2(0.0015),
                    activity_regularizer=regularizers.l1(0.0015))(x)
    
my_model = Model(inputs=model_with_dropout.input, outputs=predictions)
        
my_model.compile(optimizer='sgd',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

## Treinando o modelo

In [ ]:
my_model.fit_generator(
    train_generator,
    epochs=10, steps_per_epoch=len(X_train) / 18,
    validation_data=valid_generator, validation_steps=len(X_valid) / 18) # reali
my_model.save_weights('first_try.h5')  # Criando um modelo .h5 conseguimos dar load nos parametros
#Sem ter que ficar treinando toda vez

## Fazendo predições

In [ ]:
preds = my_model.predict(np.array(x_test), verbose=1)
sub = pd.DataFrame(preds)
col_names = one_hot.columns.values
sub.columns = col_names
sub.insert(0, 'id', df_test['id'])
sub.head(5)
sub.to_csv("submission.csv")